In [23]:
import os
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torchaudio.transforms as T_audio
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2Model, logging as hf_logging
from torch_audiomentations import Compose, AddColoredNoise, PitchShift
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import timm
import random
import glob
import torchvision.transforms as T_vision
from torch.optim.lr_scheduler import ReduceLROnPlateau
import logging

In [24]:
logging.basicConfig(filename='debug_log.txt', level=logging.DEBUG, format='%(asctime)s %(message)s')
hf_logging.set_verbosity_error()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Using device: cuda
GPU: NVIDIA GeForce RTX 3050


In [25]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [26]:
torch.cuda.empty_cache()

In [27]:
class DynamicGraphAttention(nn.Module):
    def __init__(self, in_channels, num_nodes=32, heads=8, dropout=0.1):
        super().__init__()
        self.num_nodes = num_nodes
        self.heads = heads
        self.head_dim = in_channels // heads
        self.dropout = nn.Dropout(dropout)
        self.query = nn.Linear(in_channels, in_channels)
        self.key = nn.Linear(in_channels, in_channels)
        self.value = nn.Linear(in_channels, in_channels)
        self.out = nn.Linear(in_channels, in_channels)
        self.scale = nn.Parameter(torch.tensor(self.head_dim**-0.5))
        self.pos_enc = nn.Parameter(torch.randn(1, num_nodes, in_channels))
    
    def forward(self, x):
        B, C, H, W = x.size()
        x_flat = x.flatten(2).transpose(1, 2)
        num_pixels = H * W
        N = min(self.num_nodes, num_pixels)
        norms = torch.norm(x_flat, dim=-1)
        _, indices = torch.topk(norms, N, dim=-1)
        indices_expanded = indices.unsqueeze(-1).expand(-1, -1, C)
        x_nodes = torch.gather(x_flat, 1, indices_expanded)
        if self.pos_enc.size(1) >= N:
            x_nodes = x_nodes + self.pos_enc[:, :N, :]
        q = self.query(x_nodes).view(B, N, self.heads, self.head_dim).transpose(1, 2)
        k = self.key(x_nodes).view(B, N, self.heads, self.head_dim).transpose(1, 2)
        v = self.value(x_nodes).view(B, N, self.heads, self.head_dim).transpose(1, 2)
        attn_weights = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_weights = self.dropout(attn_weights)
        attn_output = torch.matmul(attn_weights, v).transpose(1, 2).contiguous().view(B, N, C)
        out_nodes = self.out(attn_output)
        out_flat = torch.zeros_like(x_flat)
        out_flat.scatter_(1, indices_expanded, out_nodes)
        out = out_flat.transpose(1, 2).view(B, C, H, W)
        return x + out

class TemporalAttentionAggregator(nn.Module):
    def __init__(self, in_channels, num_heads=4):
        super().__init__()
        self.attention_net = nn.MultiheadAttention(embed_dim=in_channels, num_heads=num_heads, batch_first=True)
        self.norm = nn.LayerNorm(in_channels)
    
    def forward(self, x):
        if x.dim() == 4:
            B, C, H, W = x.size()
            x_flat = x.flatten(2).transpose(1, 2)
        else:
            raise ValueError("Input must be 4D tensor")
        x_norm = self.norm(x_flat)
        attn_output, _ = self.attention_net(x_norm, x_norm, x_norm)
        return torch.mean(attn_output, dim=1)

class InvertedResidual(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expansion=6):
        super().__init__()
        hidden_dim = in_channels * expansion
        self.use_residual = in_channels == out_channels and stride == 1
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim, 1, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.SiLU(inplace=True),
            nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.SiLU(inplace=True),
            nn.Conv2d(hidden_dim, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(out_channels, out_channels // 16, 1),
            nn.SiLU(),
            nn.Conv2d(out_channels // 16, out_channels, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        out = self.conv(x)
        out = out * self.se(out)
        return x + out if self.use_residual else out

In [28]:
class PestLarvaNet_Medium_Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.config = [[1, 24, 2, 1], [6, 40, 3, 2], [6, 64, 3, 2], [6, 120, 3, 2], [6, 160, 2, 1]]
        self.stem = nn.Sequential(
            nn.Conv2d(1, 24, 3, 2, 1, bias=False),
            nn.BatchNorm2d(24),
            nn.SiLU(inplace=True)
        )
        layers = []
        in_channels = 24
        for t, c, n, s in self.config:
            for i in range(n):
                stride = s if i == 0 else 1
                layers.append(InvertedResidual(in_channels, c, stride, expansion=t))
                in_channels = c
            if c == 40:
                layers.append(DynamicGraphAttention(in_channels, num_nodes=32, heads=8))
        self.body = nn.Sequential(*layers)
        self.head_conv = nn.Conv2d(160, 384, 1, bias=False)
        self.aggregator = TemporalAttentionAggregator(384, num_heads=8)
    
    def forward(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(1)
        x = self.stem(x)
        x = self.body(x)
        x = self.head_conv(x)
        x = self.aggregator(x)
        return x

class PestLarvaNet_Medium_Classifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.encoder = PestLarvaNet_Medium_Encoder()
        self.classifier = nn.Sequential(
            nn.Linear(384, 192),
            nn.SiLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(192, num_classes)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        return self.classifier(x)

In [29]:
class DualInputSSLDataset(Dataset):
    def __init__(self, wav_dir, target_sample_rate=16000, target_duration_sec=5, hop_length=512, n_mels=128, n_fft=2048):
        super().__init__()
        self.wav_dir = wav_dir
        self.target_sample_rate = target_sample_rate
        self.target_num_samples = target_sample_rate * target_duration_sec
        self.target_time_frames = (self.target_num_samples // hop_length) + 1
        self.mel_spectrogram = T_audio.MelSpectrogram(
            sample_rate=target_sample_rate,
            n_fft=n_fft,
            hop_length=hop_length,
            n_mels=n_mels,
            f_min=0.0,
            f_max=target_sample_rate // 2
        )
        self.wav_files = glob.glob(os.path.join(wav_dir, '**', '*.wav'), recursive=True)
        if not self.wav_files:
            raise FileNotFoundError(f"No valid .wav files found in directory: {wav_dir}")
        print(f"Dataset initialized with {len(self.wav_files)} valid .wav files")
    
    def __len__(self):
        return len(self.wav_files)
    
    def __getitem__(self, idx):
        wav_path = self.wav_files[idx]
        try:
            waveform, sample_rate = torchaudio.load(wav_path)
            logging.debug(f"Loaded waveform from {wav_path}: shape={waveform.shape}, min={waveform.min()}, max={waveform.max()}")
            if waveform.numel() == 0:
                logging.warning(f"Empty waveform: {wav_path}. Skipping...")
                return None
            if sample_rate != self.target_sample_rate:
                waveform = torchaudio.functional.resample(waveform, sample_rate, self.target_sample_rate)
            if waveform.shape[0] > 1:
                waveform = torch.mean(waveform, dim=0, keepdim=True)
            current_len = waveform.shape[1]
            if current_len != self.target_num_samples:
                logging.warning(f"Waveform length {current_len} does not match expected {self.target_num_samples} for {wav_path}")
                return None
            
            # Check if waveform has sufficient variance
            if torch.std(waveform) < 1e-6:
                logging.warning(f"Low variance detected in {wav_path}. Skipping...")
                return None
            
            # Normalize waveform
            waveform = (waveform - waveform.mean()) / (waveform.std() + 1e-8)
            
            # Compute Mel spectrogram on-the-fly
            spectrogram = self.mel_spectrogram(waveform)
            spectrogram = torch.log(spectrogram + 1e-9)  # Log-mel for numerical stability
            
            # Standardize spectrogram time dimension
            current_time_frames = spectrogram.shape[-1]
            if current_time_frames > self.target_time_frames:
                spectrogram = spectrogram[:, :, :self.target_time_frames]
            elif current_time_frames < self.target_time_frames:
                spectrogram = F.pad(spectrogram, (0, self.target_time_frames - current_time_frames))
            
            # Normalize spectrogram (z-score)
            spectrogram_mean = spectrogram.mean()
            spectrogram_std = spectrogram.std() + 1e-8
            spectrogram = (spectrogram - spectrogram_mean) / spectrogram_std
            logging.debug(f"Spectrogram shape: {spectrogram.shape}, min={spectrogram.min()}, max={spectrogram.max()}")
            
            return waveform, spectrogram
        except Exception as e:
            logging.error(f"Error loading item {idx} ({wav_path}): {e}. Skipping...")
            return None

In [30]:
def collate_fn_remove_none_ssl(batch):
    batch = [item for item in batch if item is not None]
    if len(batch) < 1:
        logging.warning("Batch is empty after filtering None items")
        return None
    waveforms, spectrograms = zip(*batch)
    logging.debug(f"Batch shapes: waveforms={waveforms[0].shape}, spectrograms={spectrograms[0].shape}")
    return torch.stack(waveforms), torch.stack(spectrograms)

# Custom BatchNorm1d to handle single-sample batches
class SafeBatchNorm1d(nn.BatchNorm1d):
    def forward(self, input):
        logging.debug(f"SafeBatchNorm1d input shape: {input.shape}")
        if input.size(0) == 1 and self.training:
            # Switch to eval mode for single-sample batches
            self.eval()
            out = super().forward(input)
            self.train()
            return out
        return super().forward(input)

In [31]:
class TimeShift(torch.nn.Module):
    def __init__(self, p=0.5, max_shift_percent=0.4):
        super().__init__()
        self.p = p
        self.max_shift_percent = max_shift_percent

    def forward(self, spec):
        if torch.rand(1) < self.p:
            time_steps = spec.shape[-1]
            shift_amount = int(torch.rand(1) * self.max_shift_percent * time_steps)
            if torch.rand(1) < 0.5:
                shift_amount = -shift_amount
            return torch.roll(spec, shifts=shift_amount, dims=-1)
        return spec

class SpecNoise(torch.nn.Module):
    def __init__(self, p=0.5, noise_level=0.05):
        super().__init__()
        self.p = p
        self.noise_level = noise_level

    def forward(self, spec):
        if torch.rand(1) < self.p:
            noise = torch.randn_like(spec) * self.noise_level
            return spec + noise
        return spec

In [32]:
def run_audiotwin_pretraining(supervised_checkpoint_path=None):
    def create_mlp(input_dim, hidden_dim, output_dim, num_layers):
        layers = [nn.Linear(input_dim, hidden_dim), SafeBatchNorm1d(hidden_dim), nn.ReLU(inplace=True)]
        for _ in range(num_layers - 2):
            layers.extend([nn.Linear(hidden_dim, hidden_dim), SafeBatchNorm1d(hidden_dim), nn.ReLU(inplace=True)])
        layers.append(nn.Linear(hidden_dim, output_dim))
        return nn.Sequential(*layers)
    
    class AudioTwin(nn.Module):
        def __init__(self, waveform_encoder, spectrogram_encoder, proj_dim=512, pred_dim=128):
            super().__init__()
            self.waveform_encoder = waveform_encoder
            self.spectrogram_encoder = spectrogram_encoder
            waveform_out_dim = 768
            spectrogram_out_dim = 384
            self.waveform_projector = create_mlp(waveform_out_dim, proj_dim, proj_dim, 3)
            self.spectrogram_projector = create_mlp(spectrogram_out_dim, proj_dim, proj_dim, 3)
            self.waveform_predictor = create_mlp(proj_dim, pred_dim, proj_dim, 2)
            self.spectrogram_predictor = create_mlp(proj_dim, pred_dim, proj_dim, 2)
        
        def forward(self, waveform_aug, spectrogram_aug):
            logging.debug(f"AudioTwin input shapes: waveform={waveform_aug.shape}, spectrogram={spectrogram_aug.shape}")
            if waveform_aug.dim() == 3:
                waveform_aug = waveform_aug.squeeze(1)
            h_w = self.waveform_encoder(waveform_aug).last_hidden_state.mean(dim=1)
            logging.debug(f"h_w shape: {h_w.shape}, min={h_w.min()}, max={h_w.max()}")
            z_w = F.normalize(self.waveform_projector(h_w), dim=-1)
            p_w = self.waveform_predictor(z_w)
            h_s = self.spectrogram_encoder(spectrogram_aug)
            logging.debug(f"h_s shape: {h_s.shape}, min={h_s.min()}, max={h_s.max()}")
            z_s = F.normalize(self.spectrogram_projector(h_s), dim=-1)
            p_s = self.spectrogram_predictor(z_s)
            logging.debug(f"z_w={z_w.min().item():.4f}-{z_w.max().item():.4f}, z_s={z_s.min().item():.4f}-{z_s.max().item():.4f}")
            return z_w, z_s, p_w, p_s
    
    class SimSiamLoss(nn.Module):
        def __init__(self, epsilon=1e-6):
            super().__init__()
            self.epsilon = epsilon
        
        def forward(self, p1, z2, p2, z1):
            logging.debug(f"SimSiamLoss input shapes: p1={p1.shape}, z2={z2.shape}, p2={p2.shape}, z1={z1.shape}")
            z1, z2 = z1.detach(), z2.detach()
            p1 = F.normalize(p1, dim=-1)
            p2 = F.normalize(p2, dim=-1)
            loss = (-(p1 * z2).sum(dim=1).mean() - (p2 * z1).sum(dim=1).mean()) / 2
            logging.debug(f"SimSiamLoss value: {loss.item()}")
            return loss
    
    class BarlowTwinsLoss(nn.Module):
        def __init__(self, lambda_param=5e-3, epsilon=1e-6):
            super().__init__()
            self.lambda_param = lambda_param
            self.epsilon = epsilon
        
        def forward(self, z1, z2):
            logging.debug(f"BarlowTwinsLoss input shapes: z1={z1.shape}, z2={z2.shape}")
            B, D = z1.size(0), z1.size(1)
            z1_mean = z1.mean(0)
            z2_mean = z2.mean(0)
            z1_std = z1.std(0) + self.epsilon
            z2_std = z2.std(0) + self.epsilon
            z1 = (z1 - z1_mean) / z1_std
            z2 = (z2 - z2_mean) / z2_std
            c = (z1.T @ z2) / B
            on_diag = torch.diagonal(c).add_(-1).pow_(2).sum()
            off_diag = c.pow_(2).sum() - on_diag
            loss = on_diag + self.lambda_param * off_diag
            logging.debug(f"BarlowTwinsLoss value: {loss.item()}, z1_std={z1_std.min().item():.4f}-{z1_std.max().item():.4f}, z2_std={z2_std.min().item():.4f}-{z2_std.max().item():.4f}")
            return loss
    
    # Định nghĩa các bộ siêu tham số
    hyperparameter_sets = [
        {'lambda_param': 1e-4, 'epsilon': 1e-6, 'lambda_barlow': 0.001},
        {'lambda_param': 5e-4, 'epsilon': 1e-6, 'lambda_barlow': 0.0025},
        {'lambda_param': 1e-3, 'epsilon': 1e-6, 'lambda_barlow': 0.005},
        {'lambda_param': 5e-3, 'epsilon': 1e-6, 'lambda_barlow': 0.0075},
        {'lambda_param': 1e-2, 'epsilon': 1e-6, 'lambda_barlow': 0.01}
    ]

    waveform_augment = Compose([
        AddColoredNoise(min_snr_in_db=3.0, max_snr_in_db=30.0, min_f_decay=0.0, max_f_decay=1e-6, p=0.5, sample_rate=16000, output_type='tensor'),
        PitchShift(min_transpose_semitones=-4, max_transpose_semitones=4, p=0.5, sample_rate=16000, output_type='tensor')
    ], output_type='tensor')
    
    spectrogram_augment = T_vision.Compose([
        T_vision.RandomApply([T_vision.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.5),
        T_vision.RandomAutocontrast(p=0.5),
        TimeShift(p=0.5, max_shift_percent=0.4),
        SpecNoise(p=0.5, noise_level=0.05)
    ])
    
    # Use preprocessed directory
    WAV_DIR = r"F:\SSL_Wav_Preprocessed"
    ssl_dataset = DualInputSSLDataset(wav_dir=WAV_DIR)
    ssl_loader = DataLoader(ssl_dataset, batch_size=32, shuffle=True, num_workers=0, collate_fn=collate_fn_remove_none_ssl)
    
    waveform_encoder = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
    for param in waveform_encoder.parameters():
        param.requires_grad = False
    
    spectrogram_encoder = PestLarvaNet_Medium_Encoder()

    if supervised_checkpoint_path and os.path.exists(supervised_checkpoint_path):
        print(f"Initializing AudioTwin with weights from: {supervised_checkpoint_path}")
        try:
            checkpoint_data = torch.load(supervised_checkpoint_path, map_location=device, weights_only=True)
            if 'model_state_dict' in checkpoint_data:
                full_model_state_dict = checkpoint_data['model_state_dict']
            else:
                full_model_state_dict = checkpoint_data
            
            adjusted_state_dict = {}
            for key, value in full_model_state_dict.items():
                if key.startswith(('stem.', 'body.', 'head_conv.', 'aggregator.')):
                    new_key = f"encoder.{key}"
                    adjusted_state_dict[new_key] = value
                elif key.startswith(('classifier.')):
                    continue
                else:
                    adjusted_state_dict[key] = value
            
            spectrogram_encoder.load_state_dict(adjusted_state_dict, strict=False)
            print("Successfully initialized spectrogram branch!")
        except Exception as e:
            print(f"Error loading Experiment 1 checkpoint: {e}. Continuing with random weights.")

    for idx, params in enumerate(hyperparameter_sets):
        lambda_param = params['lambda_param']
        epsilon = params['epsilon']
        lambda_barlow = params['lambda_barlow']
        run_id = f"run_{idx+1}_lp{lambda_param:.0e}_ep{epsilon:.0e}_lb{lambda_barlow:.3f}"
        CHECKPOINT_DIR = os.path.join('audiotwin_checkpoints_medium', run_id)
        os.makedirs(CHECKPOINT_DIR, exist_ok=True)
        CSV_PATH = os.path.join(CHECKPOINT_DIR, f'{run_id}_training_log.csv')
        
        print(f"\nStarting Pre-training with AudioTwin - {run_id}")
        model = AudioTwin(waveform_encoder, spectrogram_encoder).to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6, weight_decay=1e-5)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)
        scaler = torch.amp.GradScaler('cuda')
        simsiam_loss_fn = SimSiamLoss(epsilon=epsilon).to(device)
        barlow_loss_fn = BarlowTwinsLoss(lambda_param=lambda_param, epsilon=epsilon).to(device)

        EPOCHS = 100
        with open(CSV_PATH, 'w', newline='') as csvfile:
            fieldnames = ['epoch', 'avg_loss', 'avg_grad_norm', 'valid_batches', 'learning_rate']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for epoch in range(EPOCHS):
                model.train()
                total_loss = 0.0
                total_grad_norm = 0.0
                total_batches = 0
                progress_bar = tqdm(ssl_loader, desc=f"{run_id} Epoch {epoch+1}/{EPOCHS}")

                for i, data_batch in enumerate(progress_bar):
                    if data_batch is None:
                        continue
                    waveforms, spectrograms = data_batch
                    waveforms, spectrograms = waveforms.to(device), spectrograms.to(device)
                    
                    optimizer.zero_grad()
                
                    with torch.amp.autocast('cuda'):
                        waveform_aug = waveform_augment(waveforms)
                        spectrogram_aug = spectrogram_augment(spectrograms)
                        logging.debug(f"Augmented shapes: waveform={waveform_aug.shape}, spectrogram={spectrogram_aug.shape}")
                        
                        z_w, z_s, p_w, p_s = model(waveform_aug, spectrogram_aug)
                        
                        # Kiểm tra đầu ra không hợp lệ từ model trước khi tính loss
                        if torch.isnan(z_w).any() or torch.isnan(z_s).any() or torch.isinf(z_w).any() or torch.isinf(z_s).any():
                            logging.error(f"NaN/Inf detected in model outputs at batch {i}. Skipping batch.")
                            continue
                        
                        loss_simsiam = simsiam_loss_fn(p_w, z_s, p_s, z_w)
                        loss_barlow = barlow_loss_fn(z_w, z_s)
                        
                        if torch.isnan(loss_simsiam) or torch.isnan(loss_barlow):
                            logging.error(f"NaN/Inf detected in loss at batch {i}. Skipping batch.")
                            continue
                            
                        loss = loss_simsiam + lambda_barlow * loss_barlow

                    scaler.scale(loss).backward()
                    
                    scaler.unscale_(optimizer)
                    
                    # Clip gradients
                    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    
   
                    scaler.step(optimizer)
                    
                    scaler.update()
                    
                    total_loss += loss.item()
                    if torch.isfinite(grad_norm): # Chỉ ghi lại grad norm hữu hạn
                        total_grad_norm += grad_norm.item()
                    total_batches += 1
                    progress_bar.set_postfix(loss=f"{loss.item():.4f}", grad_norm=f"{grad_norm.item():.4f}")

                if total_batches > 0:
                    avg_loss = total_loss / total_batches
                    avg_grad_norm = total_grad_norm / total_batches
                    lr = optimizer.param_groups[0]['lr']
                    print(f"{run_id} Epoch {epoch+1}/{EPOCHS} | Avg Loss: {avg_loss:.4f}, Avg Grad Norm: {avg_grad_norm:.4f}, LR: {lr:.6f}")
                    writer.writerow({'epoch': epoch + 1, 'avg_loss': avg_loss, 'avg_grad_norm': avg_grad_norm, 'valid_batches': total_batches, 'learning_rate': lr})
                else:
                    print(f"{run_id} Epoch {epoch+1}/{EPOCHS} | No valid batches processed")
                    writer.writerow({'epoch': epoch + 1, 'avg_loss': 0.0, 'avg_grad_norm': 0.0, 'valid_batches': 0, 'learning_rate': optimizer.param_groups[0]['lr']})
                
                scheduler.step(avg_loss)
                
                if (epoch + 1) % 5 == 0:
                    checkpoint_path = os.path.join(CHECKPOINT_DIR, f'audiotwin_pestlarvanet_medium_epoch_{epoch+1}.pth')
                    torch.save({
                        'epoch': epoch + 1,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'scaler_state_dict': scaler.state_dict(),
                        'loss': avg_loss
                    }, checkpoint_path)
                    print(f"Checkpoint saved at '{checkpoint_path}'")

In [33]:
SUPERVISED_CHECKPOINT = r'F:\best_model_medium.pth'
run_audiotwin_pretraining(supervised_checkpoint_path=SUPERVISED_CHECKPOINT)

Dataset initialized with 28466 valid .wav files
Initializing AudioTwin with weights from: F:\best_model_medium.pth
Successfully initialized spectrogram branch!

Starting Pre-training with AudioTwin - run_1_lp1e-04_ep1e-06_lb0.001


run_1_lp1e-04_ep1e-06_lb0.001 Epoch 1/100: 100%|██████| 890/890 [20:03<00:00,  1.35s/it, grad_norm=4.3386, loss=0.2732]


run_1_lp1e-04_ep1e-06_lb0.001 Epoch 1/100 | Avg Loss: 0.4129, Avg Grad Norm: 3.8811, LR: 0.000005


run_1_lp1e-04_ep1e-06_lb0.001 Epoch 2/100: 100%|██████| 890/890 [17:17<00:00,  1.17s/it, grad_norm=6.7887, loss=0.2569]


run_1_lp1e-04_ep1e-06_lb0.001 Epoch 2/100 | Avg Loss: 0.1887, Avg Grad Norm: 4.3614, LR: 0.000005


...

run_5_lp1e-02_ep1e-06_lb0.010 Epoch 98/100 | Avg Loss: 2.6786, Avg Grad Norm: 20.0370, LR: 0.000000


run_5_lp1e-02_ep1e-06_lb0.010 Epoch 99/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 890/890 [19:37<00:00,  1.32s/it, grad_norm=19.7969, loss=2.6781]


run_5_lp1e-02_ep1e-06_lb0.010 Epoch 99/100 | Avg Loss: 2.6886, Avg Grad Norm: 19.5231, LR: 0.000000


run_5_lp1e-02_ep1e-06_lb0.010 Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 890/890 [19:48<00:00,  1.34s/it, grad_norm=28.3231, loss=2.7325]


run_5_lp1e-02_ep1e-06_lb0.010 Epoch 100/100 | Avg Loss: 2.6894, Avg Grad Norm: 19.7799, LR: 0.000000
Checkpoint saved at 'audiotwin_checkpoints_medium\run_5_lp1e-02_ep1e-06_lb0.010\audiotwin_pestlarvanet_medium_epoch_100.pth'
